In [1]:
import json
import re
from pathlib import Path
from collections import Counter
from tqdm import tqdm

print("✅ Import thành công!")

✅ Import thành công!


## 1. Định nghĩa patterns cần loại bỏ

In [2]:
# Các patterns cần loại bỏ ở đầu câu hỏi
PATTERNS_TO_REMOVE = [
    r'^Câu hỏi\s*:\s*',           # "Câu hỏi: ", "Câu hỏi :"
    r'^Câu\s+\d+\s*:\s*',         # "Câu 13: ", "Câu 1 :", "Câu 123 : "
    r'^Câu\s+\d+\.\s*',           # "Câu 9.", "Câu 12."
    r'^Câu\s+\d+\)\s*',           # "Câu 1)", "Câu 23)"
    r'^Question\s+\d+\s*:\s*',    # "Question 6 :", "Question 1:"
    r'^Question\s*:\s*',          # "Question: ", "Question :"
    r'^Q\s+\d+\s*:\s*',           # "Q 5 :", "Q 1:"
    r'^Q\s*:\s*',                 # "Q: ", "Q :"
    r'^\d+\.\s*',                 # "1. ", "123. "
    r'^\d+\)\s*',                 # "1) ", "23) "
]

def clean_question_prefix(text):
    """
    Loại bỏ các prefix thừa ở đầu câu hỏi
    """
    original_text = text
    
    for pattern in PATTERNS_TO_REMOVE:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Strip whitespace
    text = text.strip()
    
    return text, text != original_text

# Test patterns
test_cases = [
    "Câu hỏi: Đây là câu hỏi test",
    "Câu hỏi : Test với space",
    "Câu 13: Câu hỏi số 13",
    "Câu 1 : Test với space trước dấu hai chấm",
    "Câu 9. Câu hỏi với dấu chấm",
    "Câu 1) Câu hỏi với dấu ngoặc",
    "Câu 123) Test",
    "Question 6 : Test question",
    "Question 10: Another test",
    "Question: What is this?",
    "Q 5 : Quick",
    "Q: Quick question",
    "1. First question",
    "23) Question with parenthesis",
    "Đây là câu không có prefix"
]

print("🔍 Test patterns:")
print("="*80)
for test in test_cases:
    cleaned, changed = clean_question_prefix(test)
    status = "✅ CLEANED" if changed else "ℹ️  NO CHANGE"
    print(f"{status}")
    print(f"  Before: {test}")
    print(f"  After:  {cleaned}")
    print()

🔍 Test patterns:
✅ CLEANED
  Before: Câu hỏi: Đây là câu hỏi test
  After:  Đây là câu hỏi test

✅ CLEANED
  Before: Câu hỏi : Test với space
  After:  Test với space

✅ CLEANED
  Before: Câu 13: Câu hỏi số 13
  After:  Câu hỏi số 13

✅ CLEANED
  Before: Câu 1 : Test với space trước dấu hai chấm
  After:  Test với space trước dấu hai chấm

✅ CLEANED
  Before: Câu 9. Câu hỏi với dấu chấm
  After:  Câu hỏi với dấu chấm

✅ CLEANED
  Before: Câu 1) Câu hỏi với dấu ngoặc
  After:  Câu hỏi với dấu ngoặc

✅ CLEANED
  Before: Câu 123) Test
  After:  Test

✅ CLEANED
  Before: Question 6 : Test question
  After:  Test question

✅ CLEANED
  Before: Question 10: Another test
  After:  Another test

✅ CLEANED
  Before: Question: What is this?
  After:  What is this?

✅ CLEANED
  Before: Q 5 : Quick
  After:  Quick

✅ CLEANED
  Before: Q: Quick question
  After:  Quick question

✅ CLEANED
  Before: 1. First question
  After:  First question

✅ CLEANED
  Before: 23) Question with parenthesis
  After:

## 2. Load dataset

In [3]:
# Chọn file dataset cần clean
# Uncomment file bạn muốn xử lý:

# Option 1: VNHSGE dataset
input_path = Path('../data/sft_dataset_vnhsge/train_sft.jsonl')
output_path = Path('../data/sft_dataset_vnhsge/train_sft_cleaned.jsonl')

# Option 2: VMLU dataset
# input_path = Path('../data/sft_dataset_vmlu/train_sft_vmlu.jsonl')
# output_path = Path('../data/sft_dataset_vmlu/train_sft_vmlu_cleaned.jsonl')

# Option 3: Merged final dataset
# input_path = Path('../data/train_sft_final.jsonl')
# output_path = Path('../data/train_sft_final_cleaned.jsonl')

print(f"📂 Input file: {input_path}")
print(f"📂 Output file: {output_path}")

# Load data
data = []
with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

print(f"\n✅ Loaded {len(data)} samples")

📂 Input file: ..\data\sft_dataset_vnhsge\train_sft.jsonl
📂 Output file: ..\data\sft_dataset_vnhsge\train_sft_cleaned.jsonl

✅ Loaded 1573 samples


## 3. Phân tích patterns trong dataset

In [4]:
# Phân tích xem có bao nhiêu câu hỏi có prefix
patterns_found = Counter()
samples_with_prefix = []

for idx, item in enumerate(data):
    for msg in item['messages']:
        if msg['role'] == 'user':
            content = msg['content']
            # Get first line (question line)
            first_line = content.split('\n')[0]
            
            # Check each pattern
            for pattern in PATTERNS_TO_REMOVE:
                match = re.search(pattern, first_line, flags=re.IGNORECASE)
                if match:
                    patterns_found[match.group(0)] += 1
                    if len(samples_with_prefix) < 10:  # Keep first 10 examples
                        samples_with_prefix.append((idx, first_line[:100]))
                    break

print("📊 Patterns found in dataset:")
print("="*80)
if patterns_found:
    for pattern, count in patterns_found.most_common():
        print(f"  '{pattern}': {count} occurrences ({count/len(data)*100:.1f}%)")
    
    print(f"\n🔍 Examples with prefix (first 10):")
    for idx, sample in samples_with_prefix:
        print(f"  [{idx}] {sample}...")
else:
    print("  ✅ No patterns found - dataset is already clean!")

📊 Patterns found in dataset:
  'Câu 10. ': 13 occurrences (0.8%)
  'Câu 11. ': 12 occurrences (0.8%)
  'Câu 12. ': 12 occurrences (0.8%)
  'Câu 13. ': 12 occurrences (0.8%)
  'Câu 15. ': 12 occurrences (0.8%)
  'Câu 16. ': 12 occurrences (0.8%)
  'Câu 3. ': 11 occurrences (0.7%)
  'Câu 14. ': 11 occurrences (0.7%)
  'Câu 17. ': 11 occurrences (0.7%)
  'Câu 19. ': 11 occurrences (0.7%)
  'Câu 20. ': 11 occurrences (0.7%)
  'Câu 21. ': 11 occurrences (0.7%)
  'Câu 24. ': 11 occurrences (0.7%)
  'Câu 25. ': 11 occurrences (0.7%)
  'Câu 27. ': 11 occurrences (0.7%)
  'Câu 29. ': 11 occurrences (0.7%)
  'Câu 21: ': 11 occurrences (0.7%)
  'Câu 1. ': 10 occurrences (0.6%)
  'Câu 2. ': 10 occurrences (0.6%)
  'Câu 6. ': 10 occurrences (0.6%)
  'Câu 26. ': 10 occurrences (0.6%)
  'Câu 28. ': 10 occurrences (0.6%)
  'Câu 30. ': 10 occurrences (0.6%)
  'Câu 33. ': 10 occurrences (0.6%)
  'Câu 39. ': 10 occurrences (0.6%)
  'Câu 2: ': 10 occurrences (0.6%)
  'Câu 18: ': 10 occurrences (0.6%)
  'C

## 4. Preview trước và sau khi clean

In [5]:
# Preview 5 samples trước và sau khi clean
print("🔍 Preview - Trước và sau khi clean:")
print("="*80)

preview_count = 0
max_previews = 5

for idx, item in enumerate(data[:50]):  # Check first 50 samples
    if preview_count >= max_previews:
        break
    
    for msg in item['messages']:
        if msg['role'] == 'user':
            original = msg['content']
            lines = original.split('\n')
            first_line = lines[0]
            
            cleaned_first_line, changed = clean_question_prefix(first_line)
            
            if changed:
                preview_count += 1
                print(f"\n📝 Sample {idx} (ID: {item.get('id', 'N/A')}):")
                print(f"  ❌ BEFORE: {first_line[:150]}...")
                print(f"  ✅ AFTER:  {cleaned_first_line[:150]}...")
                
                if preview_count >= max_previews:
                    break

if preview_count == 0:
    print("\nℹ️  No changes needed - all questions are already clean!")

🔍 Preview - Trước và sau khi clean:

📝 Sample 0 (ID: MET_Bio_IE_2019_1):
  ❌ BEFORE: Câu 81: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2? ...
  ✅ AFTER:  Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2?...

📝 Sample 1 (ID: MET_Bio_IE_2019_2):
  ❌ BEFORE: Câu 82: Động vật nào sau đây trao đổi khí với môi trường thông qua hệ thống ống khí?  ...
  ✅ AFTER:  Động vật nào sau đây trao đổi khí với môi trường thông qua hệ thống ống khí?...

📝 Sample 2 (ID: MET_Bio_IE_2019_3):
  ❌ BEFORE: Câu 83: Axit amin là đơn phân cấu tạo nên phân tử nào sau đây? ...
  ✅ AFTER:  Axit amin là đơn phân cấu tạo nên phân tử nào sau đây?...

📝 Sample 3 (ID: MET_Bio_IE_2019_4):
  ❌ BEFORE: Câu 84: Phân tử nào sau đây trực tiếp làm khuôn cho quá trình dịch mã?  ...
  ✅ AFTER:  Phân tử nào sau đây trực tiếp làm khuôn cho quá trình dịch mã?...

📝 Sample 4 (ID: MET_Bio_IE_2019_5):
  ❌ BEFORE: Câu 85: Một phân tử ADN ở vi kh

## 5. Clean toàn bộ dataset

In [6]:
# Clean dataset
cleaned_data = []
stats = {
    'total': len(data),
    'cleaned': 0,
    'unchanged': 0
}

print("🔄 Cleaning dataset...")
for item in tqdm(data):
    cleaned_item = item.copy()
    item_changed = False
    
    for msg in cleaned_item['messages']:
        if msg['role'] == 'user':
            original_content = msg['content']
            lines = original_content.split('\n')
            
            # Clean first line (question)
            if lines:
                cleaned_first_line, changed = clean_question_prefix(lines[0])
                if changed:
                    lines[0] = cleaned_first_line
                    msg['content'] = '\n'.join(lines)
                    item_changed = True
    
    cleaned_data.append(cleaned_item)
    
    if item_changed:
        stats['cleaned'] += 1
    else:
        stats['unchanged'] += 1

print(f"\n✅ Cleaning completed!")
print(f"\n📊 Statistics:")
print(f"  Total samples: {stats['total']}")
print(f"  Cleaned: {stats['cleaned']} ({stats['cleaned']/stats['total']*100:.1f}%)")
print(f"  Unchanged: {stats['unchanged']} ({stats['unchanged']/stats['total']*100:.1f}%)")

🔄 Cleaning dataset...


100%|██████████| 1573/1573 [00:00<00:00, 66903.69it/s]


✅ Cleaning completed!

📊 Statistics:
  Total samples: 1573
  Cleaned: 1386 (88.1%)
  Unchanged: 187 (11.9%)


## 6. Save cleaned dataset

In [7]:
# Save cleaned data
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w', encoding='utf-8') as f:
    for item in cleaned_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✅ Saved cleaned dataset to: {output_path}")
print(f"📦 File size: {output_path.stat().st_size / (1024*1024):.2f} MB")
print(f"📊 Total samples: {len(cleaned_data)}")

✅ Saved cleaned dataset to: ..\data\sft_dataset_vnhsge\train_sft_cleaned.jsonl
📦 File size: 1.07 MB
📊 Total samples: 1573


## 7. Verify cleaned dataset

In [8]:
# Verify cleaned file
verify_data = []
with open(output_path, 'r', encoding='utf-8') as f:
    for line in f:
        verify_data.append(json.loads(line.strip()))

print(f"✅ Verify successful!")
print(f"📊 Loaded {len(verify_data)} samples from cleaned file")
print(f"📊 Matches original count: {len(verify_data) == len(data)}")

# Check if any patterns still exist
remaining_patterns = Counter()
for item in verify_data:
    for msg in item['messages']:
        if msg['role'] == 'user':
            first_line = msg['content'].split('\n')[0]
            for pattern in PATTERNS_TO_REMOVE:
                match = re.search(pattern, first_line, flags=re.IGNORECASE)
                if match:
                    remaining_patterns[match.group(0)] += 1
                    break

if remaining_patterns:
    print(f"\n⚠️ Warning: Some patterns still exist:")
    for pattern, count in remaining_patterns.most_common():
        print(f"  '{pattern}': {count} occurrences")
else:
    print(f"\n✅ All patterns removed successfully!")

✅ Verify successful!
📊 Loaded 1573 samples from cleaned file
📊 Matches original count: True

✅ All patterns removed successfully!


## 8. Compare samples side-by-side

In [9]:
# Show side-by-side comparison for samples that changed
print("🔍 Side-by-side comparison (samples that changed):")
print("="*80)

shown = 0
max_show = 10

for orig, cleaned in zip(data, verify_data):
    if shown >= max_show:
        break
    
    orig_user = None
    cleaned_user = None
    
    for msg in orig['messages']:
        if msg['role'] == 'user':
            orig_user = msg['content']
            break
    
    for msg in cleaned['messages']:
        if msg['role'] == 'user':
            cleaned_user = msg['content']
            break
    
    if orig_user != cleaned_user:
        shown += 1
        print(f"\n📝 Sample {shown} (ID: {orig.get('id', 'N/A')}):")
        print(f"\n  ❌ ORIGINAL:")
        print(f"     {orig_user[:200]}...")
        print(f"\n  ✅ CLEANED:")
        print(f"     {cleaned_user[:200]}...")
        print(f"\n  {'-'*76}")

if shown == 0:
    print("\nℹ️  No changes detected")

🔍 Side-by-side comparison (samples that changed):

ℹ️  No changes detected


## 9. Batch clean multiple files (Optional)

In [10]:
# Uncomment để clean nhiều files cùng lúc
"""
files_to_clean = [
    ('../data/sft_dataset_vnhsge/train_sft.jsonl', '../data/sft_dataset_vnhsge/train_sft_cleaned.jsonl'),
    ('../data/sft_dataset_vmlu/train_sft_vmlu.jsonl', '../data/sft_dataset_vmlu/train_sft_vmlu_cleaned.jsonl'),
    ('../data/train_sft_final.jsonl', '../data/train_sft_final_cleaned.jsonl'),
]

for input_file, output_file in files_to_clean:
    input_path = Path(input_file)
    output_path = Path(output_file)
    
    if not input_path.exists():
        print(f"⚠️  Skipping {input_path} - file not found")
        continue
    
    print(f"\n🔄 Processing: {input_path}")
    
    # Load
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    
    # Clean
    cleaned_data = []
    cleaned_count = 0
    
    for item in tqdm(data, desc="Cleaning"):
        cleaned_item = item.copy()
        item_changed = False
        
        for msg in cleaned_item['messages']:
            if msg['role'] == 'user':
                original_content = msg['content']
                lines = original_content.split('\n')
                
                if lines:
                    cleaned_first_line, changed = clean_question_prefix(lines[0])
                    if changed:
                        lines[0] = cleaned_first_line
                        msg['content'] = '\n'.join(lines)
                        item_changed = True
        
        cleaned_data.append(cleaned_item)
        if item_changed:
            cleaned_count += 1
    
    # Save
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in cleaned_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"  ✅ Saved to: {output_path}")
    print(f"  📊 Cleaned: {cleaned_count}/{len(data)} samples ({cleaned_count/len(data)*100:.1f}%)")
"""
print("💡 Uncomment code above để clean nhiều files cùng lúc")

💡 Uncomment code above để clean nhiều files cùng lúc


## 📋 Tổng kết

### Kết quả:
- ✅ Đã loại bỏ các prefix thừa ở đầu câu hỏi
- ✅ Đã lưu dataset đã clean
- ✅ Đã verify kết quả

### Patterns đã xử lý:
- `Câu hỏi: `
- `Câu 13: ` (Câu + số + dấu hai chấm)
- `Question: `
- `Q: `
- `1. ` (số + dấu chấm)

### File output:
- Dataset đã clean được lưu với suffix `_cleaned.jsonl`
- Giữ nguyên format và structure
- Chỉ thay đổi phần user content (câu hỏi)

### Bước tiếp theo:
1. ✅ Sử dụng file `_cleaned.jsonl` cho training
2. 🔄 Có thể merge cleaned datasets
3. 🎯 Apply thêm augmentation hoặc reasoning

### Lưu ý:
- File gốc được giữ nguyên
- Có thể dễ dàng quay lại nếu cần
- Patterns có thể tùy chỉnh trong biến `PATTERNS_TO_REMOVE`